
# Customer Churn Attribution — Reproducible Notebook

This notebook loads the four datasets, computes simple monthly features, and applies the **risk scoring** logic from the report.

> Folder layout expected:
>
> ```
> customer-churn-attribution/
> ├── data/
> │   ├── subscriptions.csv
> │   ├── feature_usage.csv
> │   ├── support_tickets.csv
> │   ├── churn_events.csv
> │   └── accounts.csv
> └── notebooks/
>     └── churn_analysis.ipynb
> ```


In [ ]:

# 1) Imports
import pandas as pd
import numpy as np

# Pretty display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)
print("✅ Libraries ready")


In [ ]:

# 2) Load datasets (graceful if files are not present)
from pathlib import Path

DATA_DIR = Path("../data")

def safe_read_csv(path):
    try:
        df = pd.read_csv(path)
        print(f"Loaded: {path.name}  shape={df.shape}")
        return df
    except FileNotFoundError:
        print(f"⚠️  Not found: {path}")
        return pd.DataFrame()

subs = safe_read_csv(DATA_DIR / "subscriptions.csv")
usage = safe_read_csv(DATA_DIR / "feature_usage.csv")
tickets = safe_read_csv(DATA_DIR / "support_tickets.csv")
churn  = safe_read_csv(DATA_DIR / "churn_events.csv")
accounts = safe_read_csv(DATA_DIR / "accounts.csv")

display(subs.head(3))
display(usage.head(3))
display(tickets.head(3))
display(churn.head(3))
display(accounts.head(3))



## Feature Engineering (monthly panel)

This section demonstrates the *deployment-ready* features used in the report:
- `usage_days_m`: distinct active days in a month
- `is_downgrade`: downgrade flag from subscriptions
- `ttr_p50_m`: median ticket response time (hours)
- `csat_m`: mean CSAT in month


In [ ]:

# 3) Basic feature engineering (example skeleton)
# NOTE: This is a lightweight example; adapt to your real schemas as needed.

# --- usage_days_m ---
usage_feat = pd.DataFrame()
if not usage.empty:
    # Expect columns like: account_id, date (YYYY-MM-DD), maybe feature_name
    use = usage.copy()
    # Ensure date type
    if "date" in use.columns:
        use["date"] = pd.to_datetime(use["date"])
        use["month"] = use["date"].dt.to_period("M").dt.to_timestamp()
        usage_feat = (
            use.groupby(["account_id", "month"])["date"]
              .nunique()
              .rename("usage_days_m")
              .reset_index()
        )
    else:
        print("⚠️  'feature_usage.csv' missing 'date' column; please add or adapt.")

# --- is_downgrade ---
downgrade_feat = pd.DataFrame()
if not subs.empty:
    # Expect columns: account_id, month_start, is_downgrade (0/1), mrr
    s = subs.copy()
    # Try to coerce month_start
    for col in ["month", "month_start", "period_start"]:
        if col in s.columns:
            s[col] = pd.to_datetime(s[col]).dt.to_period("M").dt.to_timestamp()
            s = s.rename(columns={col: "month"})
            break
    if "month" not in s.columns:
        print("⚠️  'subscriptions.csv' missing a month column; please add 'month' or 'month_start'.")
    downgrade_feat = s.groupby(["account_id","month"], as_index=False).agg({
        "is_downgrade":"max",  # if any downgrade happened in month
        "mrr":"max"            # keep one MRR value
    })

# --- tickets: ttr_p50_m & csat_m ---
tickets_feat = pd.DataFrame()
if not tickets.empty:
    # Expect columns: account_id, created_at, ttr_hours, csat
    t = tickets.copy()
    # Coerce time
    for col in ["created_at", "created", "timestamp"]:
        if col in t.columns:
            t[col] = pd.to_datetime(t[col])
            t["month"] = t[col].dt.to_period("M").dt.to_timestamp()
            break
    if "month" not in t.columns:
        print("⚠️  'support_tickets.csv' missing a timestamp column; please add 'created_at'.")
    # Standardize column names if present
    if "ttr_hours" not in t.columns:
        # Try alternate names
        for alt in ["ttr", "response_hours", "response_time_hours"]:
            if alt in t.columns:
                t = t.rename(columns={alt: "ttr_hours"})
                break
    if "csat" not in t.columns:
        for alt in ["csat_score", "customer_satisfaction"]:
            if alt in t.columns:
                t = t.rename(columns={alt: "csat"})
                break
    tickets_feat = t.groupby(["account_id","month"], as_index=False).agg(
        ttr_p50_m = ("ttr_hours","median"),
        csat_m    = ("csat","mean")
    )

# Merge monthly panel
monthly = None
to_merge = [df for df in [usage_feat, downgrade_feat, tickets_feat] if not df.empty]
if to_merge:
    from functools import reduce
    monthly = reduce(lambda l,r: pd.merge(l, r, on=["account_id","month"], how="outer"), to_merge)
    # Fill missing with safe defaults
    for col, val in [("usage_days_m", 0), ("is_downgrade", 0), ("ttr_p50_m", np.nan), ("csat_m", np.nan), ("mrr", np.nan)]:
        if col in monthly.columns:
            monthly[col] = monthly[col].fillna(val)
    print("✅ Built monthly feature table:", monthly.shape)
    display(monthly.head(10))
else:
    print("⚠️  Could not build monthly features; please check your input files.")



## Risk Scoring (from the report)

High-level rule set:

```
if usage_days_m < 6:
    if is_downgrade == 1 or ttr_p50_m > 48 or csat_m < 3.5:
        risk_score = 'High'
    else:
        risk_score = 'Medium'
else:
    risk_score = 'Low'
```


In [ ]:

# 4) Risk scoring implementation
def apply_risk_score(df):
    def score_row(row):
        usage_days = row.get("usage_days_m", 0)
        downgrade  = row.get("is_downgrade", 0)
        ttr        = row.get("ttr_p50_m", np.nan)
        csat       = row.get("csat_m", np.nan)

        if usage_days < 6:
            if (downgrade == 1) or (pd.notnull(ttr) and ttr > 48) or (pd.notnull(csat) and csat < 3.5):
                return "High"
            return "Medium"
        return "Low"

    out = df.copy()
    out["risk_score"] = out.apply(score_row, axis=1)
    return out

scored = None
if monthly is not None:
    scored = apply_risk_score(monthly)
    print("✅ Risk scoring complete:", scored.shape)
    display(scored.head(10))
else:
    print("⚠️  Monthly features not available; skipping scoring.")



## Export Deliverables

- `high_risk_accounts.csv`: all Medium/High risk accounts with key metrics  
- `high_risk_top20.csv`: top 20 high-risk accounts ranked by `MRR × Risk Level`


In [ ]:

# 5) Export deliverables (files will be written next to this notebook by default)
if scored is not None and not scored.empty:
    # Keep key columns
    keep_cols = [c for c in ["account_id","month","mrr","usage_days_m","is_downgrade","ttr_p50_m","csat_m","risk_score"] if c in scored.columns]
    export = scored[keep_cols].copy()

    # Medium/High set
    mask_mh = export["risk_score"].isin(["Medium","High"])
    mh = export[mask_mh].sort_values(["risk_score","mrr"], ascending=[True, False])
    mh.to_csv("high_risk_accounts.csv", index=False)
    print("✅ Wrote: high_risk_accounts.csv  rows=", len(mh))

    # Top-20 by MRR × Risk Level (High > Medium > Low)
    risk_rank = {"High": 2, "Medium": 1, "Low": 0}
    export["risk_weight"] = export["risk_score"].map(risk_rank).fillna(0)
    export["priority"] = export["mrr"].fillna(0) * (1 + export["risk_weight"])
    top20 = export.sort_values(["priority"], ascending=False).head(20)
    top20.to_csv("high_risk_top20.csv", index=False)
    print("✅ Wrote: high_risk_top20.csv  rows=", len(top20))

    display(top20.head(10))
else:
    print("⚠️  Nothing to export; ensure input CSVs are present and monthly features are computed.")
